In [ ]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [ ]:
# rng = np.random.RandomState(1)
# X = np.dot(rng.rand(2, 2), rng.randn(2, 200)).T
# plt.scatter(X[:, 0], X[:, 1])
# plt.axis('equal');

In [ ]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=2)
# pca.fit(X)

In [ ]:
# print(pca.components_)

In [ ]:
# print(pca.explained_variance_)



In [ ]:
# print(pca.explained_variance_ratio_)

In [ ]:


# # plot data
# plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
# for length, vector in zip(pca.explained_variance_, pca.components_):
#     v = vector * 3 * np.sqrt(length)
#     draw_vector(pca.mean_, pca.mean_ + v)
# plt.axis('equal');

In [ ]:
# plt.plot(np.cumsum(pca.explained_variance_ratio_))
# plt.xlabel('number of components')
# plt.ylabel('cumulative explained variance');

# Transformation

In [ ]:
# pca = PCA(n_components=1)
# pca.fit(X)
# X_pca = pca.transform(X)
# print("original shape:   ", X.shape)
# print("transformed shape:", X_pca.shape)

In [ ]:
# X_new = pca.inverse_transform(X_pca)
# plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
# plt.scatter(X_new[:, 0], X_new[:, 1], alpha=0.8)
# plt.axis('equal');

# Environmental Data and Global variance

In [ ]:
import xarray

In [ ]:
import cartopy.crs as ccrs

In [ ]:
filename = '/home/ucyo/Developments/big_files/IMK_MESSy______20141101_0000_6h02_hl.nc'
ds = xarray.open_dataset(filename)
mapping = {'tm1':'Temperature', 'month':'Time', 'lon':'Longitude', 'lat':'Latitude', 'lev':'Altitude',
           'um1':'Zonal Wind', 'vm1': 'Meridional Wind', 'qm1':'Specific Humidity',
           'press': 'Pressure', 'geopot':'Geopotential Height', 'time':'Time'}

In [ ]:
pos1,pos2,pos3 = 10, 45, -45

# for var in ['tm1', 'um1', 'vm1', 'qm1', 'press', 'geopot']:
for var in ['tm1']:
    # preprocessing
    obj = getattr(ds, var)[:,:,:,:].rename({'lev_2':'lev'})
    obj.name = '{} [{}]'.format(mapping[var], obj.attrs['units'])
    obj = obj.sortby('lev')
    obj.lev.attrs['positive'] = 'up'
    
    # across time
    dim = 'time'
    variance = obj.var(dim)
    g = variance.plot(x='lon', y='lat', 
                          col='lev', col_wrap=5, robust=True, transform=ccrs.PlateCarree(), edgecolor='face',
                          subplot_kws={'projection': ccrs.Orthographic(pos1, pos2)})
    for i,axs in enumerate(g.axes.ravel()):
        axs.coastlines(color='white', linewidth=.5, alpha=.5)
        axs.set_title(label="lev: {} Pa".format(int(variance['lev'].data[i])))
    g = g.axes[::-1]
    #plt.savefig('/home/ucyo/Developments/dissertation/Figures/analysis/variance-north-{}-{}-2014-11.jpg'.format(mapping[dim], mapping[var].replace(' ', '-')), dpi=300)
    plt.show();
    g = variance.plot(x='lon', y='lat', 
                          col='lev', col_wrap=5, robust=True, transform=ccrs.PlateCarree(), edgecolor='face',
                          subplot_kws={'projection': ccrs.Orthographic(pos1, pos3)})
    for i,axs in enumerate(g.axes.ravel()):
        axs.coastlines(color='white', linewidth=.5, alpha=.5)
        axs.set_title(label="lev: {} Pa".format(int(variance['lev'].data[i])))
    #plt.savefig('/home/ucyo/Developments/dissertation/Figures/analysis/variance-south-{}-{}-2014-11.jpg'.format(mapping[dim], mapping[var].replace(' ', '-')), dpi=300)
    plt.show();
    
    # across altitude
    dim = 'lev'
    variance = obj.var(dim).isel(time=slice(None,None,4))
    g = variance.plot(x='lon', y='lat', 
                          col='time', col_wrap=6, robust=True, transform=ccrs.PlateCarree(),
                          subplot_kws={'projection': ccrs.Orthographic(pos1,pos2)})
    for i,axs in enumerate(g.axes.ravel()):
        axs.coastlines(color='white', linewidth=.6, alpha=.5)
    #plt.savefig('/home/ucyo/Developments/dissertation/Figures/analysis/variance-north-{}-{}-2014-11.jpg'.format(mapping[dim], mapping[var].replace(' ', '-')), dpi=300)
    plt.show()
    g = variance.plot(x='lon', y='lat', 
                          col='time', col_wrap=6, robust=True, transform=ccrs.PlateCarree(),
                          subplot_kws={'projection': ccrs.Orthographic(pos1, pos3)})
    for i,axs in enumerate(g.axes.ravel()):
        axs.coastlines(color='white', linewidth=.6, alpha=.5)
    #plt.savefig('/home/ucyo/Developments/dissertation/Figures/analysis/variance-south-{}-{}-2014-11.jpg'.format(mapping[dim], mapping[var].replace(' ', '-')), dpi=300)
    plt.show()
    
    #across lat
    dim = 'lat'
    variance = obj.var(dim)
    g = variance.plot(x='lon', y='time', col='lev', col_wrap=5, robust=True)
    for i,axs in enumerate(g.axes.ravel()):
        try:
            axs.set_title(label="lev: {} Pa".format(int(variance['lev'].data[i])))
        except IndexError:
            axs.set_title(label="Err")
            pass
    #plt.savefig('/home/ucyo/Developments/dissertation/Figures/analysis/variance-{}-{}-2014-11.jpg'.format(mapping[dim], mapping[var].replace(' ', '-')), dpi=300)
    plt.show()
    
    #across lon
    dim = 'lon'
    variance = obj.var(dim)
    g = variance.plot(x='lat', y='time', col='lev', col_wrap=5, robust=True)
    for i,axs in enumerate(g.axes.ravel()):
        try:
            axs.set_title(label="lev: {} Pa".format(int(variance['lev'].data[i])))
        except IndexError:
            axs.set_title(label="Err")
            pass
    #plt.savefig('/home/ucyo/Developments/dissertation/Figures/analysis/variance-{}-{}-2014-11.jpg'.format(mapping[dim], mapping[var].replace(' ', '-')), dpi=300)
    plt.show()

In [ ]:
# dim = 'time'
# variance = obj.var(dim)
# # print(obj.lev.attrs)
# single = variance.sel(lev=6.5e+04, method='nearest')
# ax = plt.axes(projection=ccrs.Orthographic(-80, 35))
# single.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='face')
# ax.set_global(); ax.coastlines(color='yellow')
# # plt.savefig('../../../pasc/egu/tm1-example.svg', dpi=96, format='svg')
# plt.show();

# ax2 = plt.axes(projection=ccrs.Orthographic(100, -35))
# single.plot(ax=ax2, transform=ccrs.PlateCarree())
# ax2.set_global(); ax2.coastlines(color='gray')
# plt.show()

# Across the globe

In [ ]:
ds

In [ ]:
# dim = 'time'
# variance = obj.var(dim)
# g = variance.plot(x='lon', y='lat', 
#                       col='lev', col_wrap=5, robust=True, transform=ccrs.PlateCarree(), edgecolor='face',
#                       subplot_kws={'projection': ccrs.Orthographic(pos1, pos2)})
# # plt.gcf().suptitle('asd', y=1.05, x=.4)
# for i,axs in enumerate(g.axes.ravel()):
#     axs.coastlines(color='white', linewidth=.5, alpha=.5)
#     axs.set_title(label="lev: {} Pa".format(int(variance['lev'].data[i])))
# g = g.axes[::-1]
# plt.show();
# g = variance.plot(x='lon', y='lat', 
#                       col='lev', col_wrap=5, robust=True, transform=ccrs.PlateCarree(), edgecolor='face',
#                       subplot_kws={'projection': ccrs.Orthographic(pos1, pos3)})
# for i,axs in enumerate(g.axes.ravel()):
#     axs.coastlines(color='white', linewidth=.5, alpha=.5)
#     axs.set_title(label="lev: {} Pa".format(int(variance['lev'].data[i])))
# plt.show();

In [ ]:
# dim = 'lev'
# variance = obj.var(dim).isel(time=slice(None,None,4))
# g = variance.plot(x='lon', y='lat', 
#                       col='time', col_wrap=6, robust=True, transform=ccrs.PlateCarree(),
#                       subplot_kws={'projection': ccrs.Orthographic(pos1,pos2)})
# for i,axs in enumerate(g.axes.ravel()):
#     axs.coastlines(color='white', linewidth=.6, alpha=.5)
# plt.show()
# g = variance.plot(x='lon', y='lat', 
#                       col='time', col_wrap=6, robust=True, transform=ccrs.PlateCarree(),
#                       subplot_kws={'projection': ccrs.Orthographic(pos1, pos3)})
# for i,axs in enumerate(g.axes.ravel()):
#     axs.coastlines(color='white', linewidth=.6, alpha=.5)
# plt.show()

In [ ]:
# dim = 'lat'
# variance = obj.var(dim)
# g = variance.plot(x='lon', y='time', col='lev', col_wrap=5, robust=True)
# for i,axs in enumerate(g.axes.ravel()):
#     try:
#         axs.set_title(label="lev: {} Pa".format(int(variance['lev'].data[i])))
#     except IndexError:
#         axs.set_title(label="Err")
#         pass
# plt.show()

In [ ]:
# dim = 'lon'
# variance = obj.var(dim)
# g = variance.plot(x='lat', y='time', col='lev', col_wrap=5, robust=True)
# for i,axs in enumerate(g.axes.ravel()):
#     try:
#         axs.set_title(label="lev: {} Pa".format(int(variance['lev'].data[i])))
#     except IndexError:
#         axs.set_title(label="Err")
#         pass
# plt.show()

# As line plots